In [1]:
#Import modules for Data Exploration
import pandas as pd
from rates import CTR #In src there is a rates.py where I have developed a function to quickly calculate CTR

## Data Exploration

In [3]:
df = pd.read_csv('../input/events_log.csv.gz', compression='gzip', error_bad_lines=False)
df.head()

,uuid,timestamp,session_id,group,action,checkin,page_id,n_results,result_position
0,00000736167c507e8ec225bd9e71f9e5,2.016030e+13,78245c2c3fba013a,b,searchResultPage,NaN,cbeb66d1bc1f1bc2,5.0,NaN
1,00000c69fe345268935463abbfa5d5b3,2.016031e+13,c559c3be98dca8a4,a,searchResultPage,NaN,eb658e8722aad674,10.0,NaN
2,00003bfdab715ee59077a3670331b787,2.016030e+13,760bf89817ce4b08,a,checkin,30.0,f99a9fc1f7fdd21e,NaN,NaN
3,0000465cd7c35ad2bdeafec953e08c1a,2.016030e+13,fb905603d31b2071,a,checkin,60.0,e5626962a6939a75,NaN,10.0
4,000050cbb4ef5b42b16c4d2cf69e6358,2.016030e+13,c2bf5e5172a892dc,a,checkin,30.0,787dd6a4c371cbf9,NaN,NaN


In [4]:
df.shape

(400165, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400165 entries, 0 to 400164
Data columns (total 9 columns):
uuid               400165 non-null object
timestamp          400165 non-null float64
session_id         400165 non-null object
group              400165 non-null object
action             400165 non-null object
checkin            223824 non-null float64
page_id            400165 non-null object
n_results          136234 non-null float64
result_position    230482 non-null float64
dtypes: float64(4), object(5)
memory usage: 27.5+ MB


In [6]:
#Checking different sessions
df['session_id'].value_counts()

b254341e78af2f1a    484
593638dbb24d903a    305
549c7d24637bc68c    250
4264985570c2b41d    234
b1b9ad0b289027e1    223
                   ... 
8a0c68a4f445af85      1
020777bdfc251cd4      1
f34f50b40807b8f3      1
59d298aee940c999      1
a1bf552b4a1fbcd6      1
Name: session_id, Length: 68028, dtype: int64

In [7]:
#Checking different pages
df['page_id'].value_counts()

ffeae9d12ad83b25    24
5776ccc6144ff777    20
6ea77e97a39691d2    19
6151bd29e99dae15    19
e6fcfc833df5c687    19
                    ..
82e3ba8415a243a2     1
d4b4421037e41ed8     1
07293eb1d098ed7d     1
72ad6eeccb2f3ee0     1
0d68653f9c2a49dc     1
Name: page_id, Length: 176371, dtype: int64

## CTR calculated by Sessions Tagged as “visitPage”

Previous to developed a function, checking the number of clicks and web visits. Using CTR function where it's used the total number of clicks from sessions tagged as "visitPage" and the total amount
of sessions tracked in the dataset

In [8]:
#Calculate the number of clicks, considering each click as a register tagged with "visitPage"
print('The total amount of clicks is ', df[(df['action'] == 'visitPage')]['action'].count())

The total amount of clicks is  40107


In [9]:
#Calculate the number of page visits
print('The total amount of web visits is ', df.shape[0])

The total amount of web visits is  400165


In [10]:
#Checking there is no duplicated registers, therefore the total number of page visits could be considering all dataset registers
df.duplicated().sum()

0

In [11]:
#Calculate CTR using the function
ctr = CTR(df,'action', 'visitPage')
print('CTR based on the visitPage tag is up to ', ctr,'%')

CTR based on the visitPage tag is up to  10.02 %


### A and B groups CTR

Calculating CTR for each group. Grouping dataset based on a or b group and calculating CTR using same formula as previous to compare which group performs better.

In [12]:
#'A' group dataset
dfa = df[(df['group'] == 'a')]
print('CTR for A group is ', CTR(dfa, 'action', 'visitPage'), '%')

CTR for A group is  11.12 %


In [13]:
#'B' group dataset
dfb = df[(df['group'] == 'b')]
print('CTR for B group is ', CTR(dfb, 'action', 'visitPage'), '%')

CTR for B group is  6.76 %


### Daily CTR

Grouping registers by day and calculating CTR for each day

In [14]:
#Casting timestamp to datetime format with datetime method
df['timestamp'] = pd.to_datetime(df.timestamp, format = "%Y%m%d").dt.strftime('%Y-%m-%d')

ValueError: unconverted data remains: 103842

In [15]:
#Casting timestamp serie using timestamp method
df['timestamp'] = pd.to_datetime(df.timestamp, unit='s') 

OutOfBoundsDatetime: cannot convert input 20160301103842.0 with the unit 's'

In [16]:
df['timestamp'] = pd.Timestamp(df.timestamp)

TypeError: Cannot convert input [0         2.016030e+13
1         2.016031e+13
2         2.016030e+13
3         2.016030e+13
4         2.016030e+13
              ...     
400160    2.016030e+13
400161    2.016031e+13
400162    2.016030e+13
400163    2.016031e+13
400164    2.016030e+13
Name: timestamp, Length: 400165, dtype: float64] of type <class 'pandas.core.series.Series'> to Timestamp

In [17]:
#Issue with the serie timestamp because it's a float serie with 16 units that caused an overflow error. 
#Removing last two units
df["timestamp"]=df["timestamp"].apply(lambda x: str(round(x,0))[:-2])

In [18]:
#Casting to datetime 
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [19]:
df.index = df.timestamp

In [20]:
df.resample('D').count()

,uuid,timestamp,session_id,group,action,checkin,page_id,n_results,result_position
timestamp,,,,,,,,,
2016-03-01,59324,59324,59324,59324,59324,34296,59324,18374,28980
2016-03-02,60322,60322,60322,60322,60322,34728,60322,18902,29777
2016-03-03,60644,60644,60644,60644,60644,34911,60644,19159,31662
2016-03-04,45746,45746,45746,45746,45746,24777,45746,16675,29067
2016-03-05,36242,36242,36242,36242,36242,19715,36242,13204,23037
2016-03-06,41200,41200,41200,41200,41200,22830,41200,14612,26580
2016-03-07,52558,52558,52558,52558,52558,28833,52558,19011,33547
2016-03-08,44129,44129,44129,44129,44129,23734,44129,16297,27832


In [21]:
#Calculate number of clicks per day
dfclicks = df[(df['action'] == 'visitPage')]['action'].resample('D').count()

In [22]:
#Calculate number of visits per day
dfvisits = df['action'].resample('D').count()

In [23]:
#Create a new dataframe to store the results
dfresults = pd.DataFrame()

In [24]:
#Adding new serie to store CTR per day
dfresults['overall ctr'] = round((dfclicks/dfvisits)*100,2)
dfresults.head()

,overall ctr
timestamp,
2016-03-01,11.22
2016-03-02,11.09
2016-03-03,10.84
2016-03-04,9.39
2016-03-05,9.17


### Calculating CTR for 'A' and 'B' group and adding to the dataframe

In [28]:
#Clicks - A group
dfclicksA = df[(df['action'] == 'visitPage') & (df['group'] == 'a')]['action'].resample('D').count()

In [29]:
#Clicks - B group
dfclicksB = df[(df['action'] == 'visitPage') & (df['group'] == 'b')]['action'].resample('D').count()

In [32]:
#Visits - A group
dfvisitsA = df[(df['group'] == 'a')]['action'].resample('D').count()

In [33]:
#Visits - B group
dfvisitsB = df[(df['group'] == 'b')]['action'].resample('D').count()

In [34]:
#Calculating CTR - A group
dfresults['ctrA'] = round((dfclicksA/dfvisitsA)*100, 2)

In [35]:
#Calculating CTR - B group
dfresults['ctrB'] = round((dfclicksB/dfvisitsB)*100, 2)